In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
# 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 기본값으로 설정 (처음 5개와 마지막 5개 행만 출력)
pd.reset_option('display.max_rows')

# 출력 포맷 설정 (소수점 4자리까지)
pd.options.display.float_format = '{:.4f}'.format

import platform
import seaborn as sns

import matplotlib.pyplot as plt

# 운영 체제 확인
if platform.system() == 'Darwin':  # Mac
    print('apple gothic')
    font_name = 'AppleGothic'
elif platform.system() == 'Windows':  # Windows
    font_name = 'NanumGothic'
else:
    font_name = None

# 한글 폰트 설정
if font_name:
    plt.rcParams['font.family'] = font_name

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

apple gothic


## 데이터 로드

In [2]:
# label + 재무변수

# df = pd.read_csv('../data/5_데이터셋생성_최종.csv', dtype = {'거래소코드' : 'object'})
df = pd.read_csv('../data/5_데이터셋생성_재무.csv', dtype = {'거래소코드' : 'object'})
df.shape

(1385, 172)

In [3]:
df.columns[-10:]

Index(['비유동자산회전율', '유형자산회전율', '재고자산회전율', '매출채권회전율', '매입채무회전율', '차입금 대 매출액',
       'label', '시가총액', '상장주식수', '배당수익률'],
      dtype='object')

In [4]:
# 거시변수
# data/raw/거시경제지표.xlsx
df_eco = pd.read_excel('../data/raw/거시경제지표.xlsx')
df_eco.head()

,DATE,DGS10,DGS1,DGS6MO,T10Y2Y,WTI_oil,Dubai_oil,realGDP_usa,real_PCE,Core_PCE,CPI_sticky,Core_CPI,manuf_PMI(R),non_manuf_PMI(R),manuf_GB,non_manuf_GB,Equip_inv,TB_rtn(1y),TB_rtn(10y),IR_sm,DXY,CB_spread(AA-),CB_spread(BBB-),경제고통지수,경상수지,수입금액지수,수입물량지수,수출금액지수,수출물량지수,미국수입금액,중국수입금액,미국수출금액,중국수출금액,실업률,실업률_증감,소비자물가상승률,근원물가상승률,근원물가상승률_식품에너지제외,GDP_growth,CLI(경기선행),CCI(경기동행),NSI(=뉴스심리지수),기업실사BSI_실적,기업실사BSI_전망
0,2010-12-01,3.0919,0.3055,0.1892,2.4155,79.4275,27.7630,2.6952,1.9060,1.4491,NaN,NaN,57.0000,57.1000,호황,호황,13.4000,2.9474,4.7665,5.6600,-4.7582,0.9431,6.9640,6.7000,NaN,31.1000,19.6000,30.0000,25.5000,22.7000,23.1000,29.6000,21.6000,3.7000,NaN,3.0000,1.7000,1.7000,NaN,NaN,NaN,-1.4748,6,8
1,2011-12-01,2.6747,0.1739,0.0920,2.2426,95.0767,35.0470,1.5644,1.7015,1.5863,177.6012,0.6257,53.9000,52.6000,호황,호황,1.3000,3.4182,4.1946,5.9900,-4.9533,0.7911,6.7621,7.4000,-0.4047,14.3000,3.3000,9.2000,9.4000,2.5000,4.8000,5.7000,5.6000,3.2000,-0.5000,4.2000,3.6000,2.7000,NaN,NaN,NaN,-13.5810,-13,-10
2,2012-12-01,1.7274,0.1675,0.1282,1.4629,94.2008,3.3083,2.2891,1.3673,1.8500,-139.7270,0.7093,50.7000,56.1000,호황,호황,-9.4000,3.1237,3.4437,5.2200,2.5615,0.6325,6.2021,4.5000,1.9324,-5.6000,-1.7000,-2.4000,-1.1000,-16.6000,-6.3000,-1.0000,2.7000,3.1000,-0.1000,1.4000,1.1000,1.3000,0.0198,-0.1000,-0.5000,1.8700,-14,-17
3,2013-12-01,2.2511,0.1256,0.0846,1.9570,97.9358,-3.2022,2.1178,1.7439,1.4944,-12.9539,-0.2265,57.0000,53.0000,호황,호황,7.0000,2.6557,3.2767,4.8400,0.2399,0.3976,6.0431,4.3000,0.5835,3.2000,5.9000,4.2000,4.5000,23.2000,10.2000,13.1000,8.3000,3.2000,0.1000,1.1000,1.9000,1.6000,0.0724,0.5000,0.0000,-0.4390,8,11
4,2014-12-01,2.4325,0.1160,0.0619,1.9899,93.2583,-8.6080,2.5238,2.8306,1.4977,19.3358,0.0321,55.5000,56.2000,호황,호황,15.3000,2.4365,3.1827,4.2600,2.2457,0.3945,6.1205,4.4000,0.0747,-1.3000,11.1000,3.9000,12.5000,3.2000,21.6000,21.1000,1.4000,3.6000,0.4000,0.8000,1.6000,1.4000,0.0831,-0.3000,-0.9000,1.9345,-3,-3


In [5]:
df_eco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 44 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              14 non-null     datetime64[ns]
 1   DGS10             14 non-null     float64       
 2   DGS1              14 non-null     float64       
 3   DGS6MO            14 non-null     float64       
 4   T10Y2Y            14 non-null     float64       
 5   WTI_oil           14 non-null     float64       
 6   Dubai_oil         14 non-null     float64       
 7   realGDP_usa       14 non-null     float64       
 8   real_PCE          14 non-null     float64       
 9   Core_PCE          14 non-null     float64       
 10  CPI_sticky        13 non-null     float64       
 11  Core_CPI          13 non-null     float64       
 12  manuf_PMI(R)      14 non-null     float64       
 13  non_manuf_PMI(R)  14 non-null     float64       
 14  manuf_GB          14 non-nul

In [6]:
df_eco['year'] = df_eco['DATE'].dt.year
df_eco['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023], dtype=int32)

In [7]:
df_eco['month'] = df_eco['DATE'].dt.month

In [8]:
df_eco.isna().sum()

DATE                0
DGS10               0
DGS1                0
DGS6MO              0
T10Y2Y              0
WTI_oil             0
Dubai_oil           0
realGDP_usa         0
real_PCE            0
Core_PCE            0
CPI_sticky          1
Core_CPI            1
manuf_PMI(R)        0
non_manuf_PMI(R)    0
manuf_GB            0
non_manuf_GB        0
Equip_inv           0
TB_rtn(1y)          0
TB_rtn(10y)         0
IR_sm               0
DXY                 0
CB_spread(AA-)      0
CB_spread(BBB-)     0
경제고통지수              0
경상수지                1
수입금액지수              0
수입물량지수              0
수출금액지수              0
수출물량지수              0
미국수입금액              0
중국수입금액              0
미국수출금액              0
중국수출금액              0
실업률                 0
실업률_증감              1
소비자물가상승률            0
근원물가상승률             0
근원물가상승률_식품에너지제외     0
GDP_growth          4
CLI(경기선행)           4
CCI(경기동행)           4
NSI(=뉴스심리지수)        0
기업실사BSI_실적          0
기업실사BSI_전망          0
year                0
month     

In [9]:
df_eco_12 = df_eco[df_eco['month']==12]
df_eco_12.isna().sum()

DATE                0
DGS10               0
DGS1                0
DGS6MO              0
T10Y2Y              0
WTI_oil             0
Dubai_oil           0
realGDP_usa         0
real_PCE            0
Core_PCE            0
CPI_sticky          1
Core_CPI            1
manuf_PMI(R)        0
non_manuf_PMI(R)    0
manuf_GB            0
non_manuf_GB        0
Equip_inv           0
TB_rtn(1y)          0
TB_rtn(10y)         0
IR_sm               0
DXY                 0
CB_spread(AA-)      0
CB_spread(BBB-)     0
경제고통지수              0
경상수지                1
수입금액지수              0
수입물량지수              0
수출금액지수              0
수출물량지수              0
미국수입금액              0
중국수입금액              0
미국수출금액              0
중국수출금액              0
실업률                 0
실업률_증감              1
소비자물가상승률            0
근원물가상승률             0
근원물가상승률_식품에너지제외     0
GDP_growth          4
CLI(경기선행)           4
CCI(경기동행)           4
NSI(=뉴스심리지수)        0
기업실사BSI_실적          0
기업실사BSI_전망          0
year                0
month     

In [10]:
# 23년 데이터
df_eco_12[df_eco_12.isna().any(axis=1)]

,DATE,DGS10,DGS1,DGS6MO,T10Y2Y,WTI_oil,Dubai_oil,realGDP_usa,real_PCE,Core_PCE,CPI_sticky,Core_CPI,manuf_PMI(R),non_manuf_PMI(R),manuf_GB,non_manuf_GB,Equip_inv,TB_rtn(1y),TB_rtn(10y),IR_sm,DXY,CB_spread(AA-),CB_spread(BBB-),경제고통지수,경상수지,수입금액지수,수입물량지수,수출금액지수,수출물량지수,미국수입금액,중국수입금액,미국수출금액,중국수출금액,실업률,실업률_증감,소비자물가상승률,근원물가상승률,근원물가상승률_식품에너지제외,GDP_growth,CLI(경기선행),CCI(경기동행),NSI(=뉴스심리지수),기업실사BSI_실적,기업실사BSI_전망,year,month
0,2010-12-01,3.0919,0.3055,0.1892,2.4155,79.4275,27.7630,2.6952,1.9060,1.4491,NaN,NaN,57.0000,57.1000,호황,호황,13.4000,2.9474,4.7665,5.6600,-4.7582,0.9431,6.9640,6.7000,NaN,31.1000,19.6000,30.0000,25.5000,22.7000,23.1000,29.6000,21.6000,3.7000,NaN,3.0000,1.7000,1.7000,NaN,NaN,NaN,-1.4748,6,8,2010,12
1,2011-12-01,2.6747,0.1739,0.0920,2.2426,95.0767,35.0470,1.5644,1.7015,1.5863,177.6012,0.6257,53.9000,52.6000,호황,호황,1.3000,3.4182,4.1946,5.9900,-4.9533,0.7911,6.7621,7.4000,-0.4047,14.3000,3.3000,9.2000,9.4000,2.5000,4.8000,5.7000,5.6000,3.2000,-0.5000,4.2000,3.6000,2.7000,NaN,NaN,NaN,-13.5810,-13,-10,2011,12
12,2022-12-01,2.8257,2.6782,2.4056,-0.0341,94.7867,38.1769,1.9355,2.5319,5.2428,-20.7292,3.0293,48.4000,49.6000,불황,불황,3.2000,2.6454,3.3675,5.7600,8.2640,0.9560,6.8068,8.0000,-0.6969,-2.4000,-1.1000,-12.3000,-6.4000,19.2000,-14.6000,6.4000,-27.0000,3.0000,-0.4000,5.0000,4.7000,4.0000,NaN,NaN,NaN,-26.0955,-24,-19,2022,12
13,2023-12-01,3.8067,4.8863,5.0804,-0.6000,77.6358,-14.0871,2.5309,2.2025,4.0991,-110.0370,0.1582,47.4000,50.6000,불황,호황,-5.8000,3.5328,3.6406,5.3100,-0.5202,0.8208,7.1944,6.4000,0.3740,-11.8000,-7.1000,3.2000,6.2000,-7.6000,-7.8000,20.7000,-3.0000,3.2000,0.2000,3.2000,3.1000,2.8000,NaN,NaN,NaN,19.5655,-1,-1,2023,12


In [11]:
df_eco_final = df_eco_12[(df_eco_12['year']>=2012) & (df_eco_12['year']<=2022)]
df_eco_final.shape

(11, 46)

## 재무 + 거시 결합

In [12]:
df['결산년도']

0       2012
1       2012
2       2013
3       2015
4       2012
        ... 
1380    2020
1381    2015
1382    2017
1383    2018
1384    2021
Name: 결산년도, Length: 1385, dtype: int64

In [13]:
df_eco_final['year']

2     2012
3     2013
4     2014
5     2015
6     2016
7     2017
8     2018
9     2019
10    2020
11    2021
12    2022
Name: year, dtype: int32

In [14]:
df_merge = pd.merge(
    df, df_eco_final,
    left_on = '결산년도', right_on='year',
    how = 'left',
    suffixes=('', '_eco')
)

df_merge.shape, df.shape, df_eco_final.shape

((1385, 218), (1385, 172), (11, 46))

In [15]:
df_merge.columns[df_merge.isna().sum()>0]

Index(['상장폐지일'], dtype='object')

In [16]:
# 중복컬럼 없음
df_merge.columns[df_merge.columns.str.endswith('_eco')]

Index([], dtype='object')

In [17]:
df_merge.head()

,거래소코드,회사명,회계년도,상장일,상장폐지일,결산년도,상장년도,market,자산,유형자산,유동자산,재고자산,자본,매출액,매출원가,판매비와 관리비(물류원가 등 포함),기타(영업)비용,당기제조원가,급여,퇴직급여,상품매출원가,제품매출원가,감가상각비,대손상각비,개발비상각,기타무형자산상각비,외환차익1,외화환산이익1,외환차손1,외화환산손실1,외환차익2,외화환산이익2,외환차손2,외환환산손실2,무형자산,건설중인자산,토지,자본잉여금,이익잉여금(결손금),미처분이익잉여금(결손금),기타임의적립금,(연차배당),(중간배당),배당금지급(-),단기차입금,매입채무 및 기타유동채무,매출채권 및 기타유동채권,비유동자산,사채,유동금융자산,유동부채,유동성장기부채,장기매입채무 및 기타비유동채무,장기매출채권 및 기타비유동채권,장기차입금,현금및현금성자산,영업활동으로 인한 현금흐름,유형자산의 증가,무형자산의 증가,연구개발비,종업원수,비유동부채,부채,유형자산의증가,유형자산의감소,(투자활동으로 인한 현금유출액),투자활동으로 인한 현금유입액,자본금,관계기업 등 지분관련 투자자산,영업손익,이자보상배율,년도차,자산_전기,자산증가율,유형자산_전기,유형자산증가율,유동자산_전기,유동자산증가율,재고자산_전기,재고자산증가율,자본_전기,자본증가율,매출액_전기,매출액증가율,자산평균,유형자산평균,무형자산_전기,무형자산평균,건설중인자산_전기,건설중인자산평균,토지_전기,토지평균,자본평균,총자산회전율,감가상각율,배당률,판관비체크,순외환손익대비매출액,매출원가대비매출액,당기총제조비용,재고조정중의고정비,고정비,총비용,변동비,변동비대비매출액,고정비대비매출액,인건비대비매출액,인건비대비영업총비용,1인당매출액,1인당인건비,재고자산평균,비유동자산_전기,비유동자산평균,매출채권 및 기타유동채권평균,장기매출채권 및 기타비유동채권평균,매입채무 및 기타유동채무평균,장기매입채무 및 기타비유동채무평균,주주이익(버핏),순운전자본(민식),유무형자산,영업현금흐름대비투하자본,ROTCE현금흐름대체,유형자산대비현금흐름,유무형자산대비현금흐름,매출대비고정자산,단기차입금_전기,단기차입금평균,장기차입금_전기,장기차입금평균,유동비율,현금비율,당좌비율,자기자본비율,비유동비율,부채비율,유동부채비율,비유동부채비율,비유동장기적합률,차입금의존도,매출채권대비매입채무,순운전자본,순운전자본대비총자본,차입금대비매출액,매출채권대비매입채무_before,영업활동현금흐름 대 총자산,영업활동현금흐름 대 매출액,투자안정성비율,영업활동현금흐름 대 투자활동현금지출,경영자산회전율분모,매출채권,매입채무,차입금,경영자산회전율분모_전기,매출채권_전기,매입채무_전기,차입금_전기,부채_전기,자본금_전기,영업활동현금흐름 대 총부채,자기자본회전율,자본금회전율,경영자산회전율,비유동자산회전율,유형자산회전율,재고자산회전율,매출채권회전율,매입채무회전율,차입금 대 매출액,label,시가총액,상장주식수,배당수익률,DATE,DGS10,DGS1,DGS6MO,T10Y2Y,WTI_oil,Dubai_oil,realGDP_usa,real_PCE,Core_PCE,CPI_sticky,Core_CPI,manuf_PMI(R),non_manuf_PMI(R),manuf_GB,non_manuf_GB,Equip_inv,TB_rtn(1y),TB_rtn(10y),IR_sm,DXY,CB_spread(AA-),CB_spread(BBB-),경제고통지수,경상수지,수입금액지수,수입물량지수,수출금액지수,수출물량지수,미국수입금액,중국수입금액,미국수출금액,중국수출금액,실업률,실업률_증감,소비자물가상승률,근원물가상승률,근원물가상승률_식품에너지제외,GDP_growth,CLI(경기선행),CCI(경기동행),NSI(=뉴스심리지수),기업실사BSI_실적,기업실사BSI_전망,year,month
0,000040,케이알모터스(주),2012/12,1976/05/25,NaN,2012,1976,KOSPI,119327267.0000,55810192.0000,61684604.0000,24763334.0000,60797171.0000,97926654.0000,92177389.0000,11948008.0000,0.0000,0.0000,2579833.0000,350950.0000,0.0000,0.0000,35985.0000,1248799.0000,0.0000,650714.0000,155998.0000,0.0000,470896.0000,1310924.0000,0.0000,0.0000,0.0000,0.0000,1118203.0000,0.0000,42542501.0000,3884892.0000,7235571.0000,12088682.0000,0.0000,0.0000,0.0000,0.0000,10401663.0000,28964045.0000,26201155.0000,57642663.0000,0.0000,100000.0000,43762954.0000,0.0000,407646.0000,342768.0000,0.0000,10236751.0000,-4565749.0000,937241.0000,42700.0000,0.0000,249.0000,14767142.0000,58530096.0000,937241.0000,21084.0000,992041.0000,464863.0000,59670690.0000,0.0000,-6198743.0000,-6.4700,1.0000,133337774.0000,-10.5075,57761033.0000,-3.3774,73022140.0000,-15.5262,24384487.0000,1.5536,68764654.0000,-11.5866,127720215.0000,-23.3272,126332520.5000,56785612.5000,1804381.0000,1461292.0000,0.0000,0.0000,42542501.0000,42542501.0000,64780912.5000,0.7752,0.0972,-0.0000,True,-13.6075,94.1290,14842806.0000,7635041.5537,11035931.0537,104125397.0000,93089465.9463,95.0604,11.2696,2.9928,24.5295,393280.0000,10361.0000,24573910.5000,60315635.0000,58979149.0000,26201155.0000,342768.0000,28964045.0000,407646.0000,-4858991.0000,51117833.5000,58246904.5000,0.0000,0.0000,0.0000,0.0000,0.0000,10304725.0000,10353194.0000,0.0000,0.0000,140.9516,23.3914,23.6199,50.9499,94.8114,96.2711,71.9819,24.2892,76.2829,8.7169,90.3725,17921650.0000,15.0189,10.5724,90.3725,0.0000,0.0000,0.0000,0.0000,119327267.0000,26543923.0000,29371691.0000,10401663.0000,133337774.0000,33214201.0000,35842471.0000,10304725.0000,64573121.0000,59670690.0000,0.0000,1.5117,1.6411,0.7752,1.6604,1.7245,3.9850,3.2774,3.0032,10.5724,0.0000,58954641.2260,119341379.0000,-0.0000,2012-12-01,1.7274,0.1675,0.1282,1.4629,94.2008,3.3083,2.2891,1.3673,1.8500,-139.7270,0.7093,50.7000,56.1000,호황,호황,-9.4000,3.1237,3.4437,5.2200,2.5615,0.6325,6.2021,4.5000,1.9324,-5.6000,-1.7000,-2.4000,-1.1000,-16.6000,-6.3000,-1.0000,2.7000,3.1000,-0.1000,1.4000,1.1000,1.3000,0.0198,-0.1000,-0.5000,1.8700,-14,-17,2012,12
1,000050,(주)경방,2012/12,1956/03/03,NaN,2012,1956,KOSPI,1335039930.0000,324

In [18]:
df_eco_final.drop(columns = ['DATE', 'year', 'month']).describe()

,DGS10,DGS1,DGS6MO,T10Y2Y,WTI_oil,Dubai_oil,realGDP_usa,real_PCE,Core_PCE,CPI_sticky,Core_CPI,manuf_PMI(R),non_manuf_PMI(R),Equip_inv,TB_rtn(1y),TB_rtn(10y),IR_sm,DXY,CB_spread(AA-),CB_spread(BBB-),경제고통지수,경상수지,수입금액지수,수입물량지수,수출금액지수,수출물량지수,미국수입금액,중국수입금액,미국수출금액,중국수출금액,실업률,실업률_증감,소비자물가상승률,근원물가상승률,근원물가상승률_식품에너지제외,GDP_growth,CLI(경기선행),CCI(경기동행),NSI(=뉴스심리지수),기업실사BSI_실적,기업실사BSI_전망
count,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,11.0000,10.0000,10.0000,10.0000,11.0000,11.0000,11.0000
mean,2.0332,0.8897,0.7996,0.9795,68.3669,3.8922,2.2826,2.5059,2.0904,-5.5978,0.3597,54.2636,55.9182,3.8000,1.8774,2.4884,4.1164,2.7216,0.5923,6.5710,5.1273,0.1868,3.3818,3.5455,2.1455,2.9727,6.7545,6.0273,7.2091,0.6273,3.4545,-0.0182,1.6727,1.8182,1.6727,0.0390,0.1200,-0.0100,-1.2040,-1.0000,-1.3636
std,0.5838,0.9616,0.9199,0.6879,22.7573,32.5959,1.8434,2.5129,1.2278,64.7028,0.9394,3.7850,3.0476,8.8005,0.7438,0.7204,0.8480,4.0757,0.2550,0.4159,1.2362,0.6674,14.3495,4.9092,10.1795,5.4591,14.6014,14.6079,11.4233,14.2762,0.3174,0.3894,1.3828,1.1170,0.9737,0.0493,0.8230,1.0429,11.3333,11.4804,10.8376
min,0.8519,0.1000,0.0615,-0.0341,39.2275,-47.2533,-2.2135,-2.5221,1.2301,-139.7270,-0.3626,48.2000,49.6000,-13.8000,0.8403,1.4994,2.8900,-2.2790,0.2900,6.0431,4.2000,-0.6969,-20.0000,-3.3000,-13.1000,-6.4000,-16.6000,-15.8000,-7.8000,-27.0000,3.0000,-0.8000,0.6000,0.7000,0.6000,-0.0429,-1.8000,-1.5000,-26.0955,-24.0000,-19.0000
25%,1.7437,0.1466,0.1064,0.4230,49.7862,-13.6125,2.0267,1.8728,1.4960,-20.8943,-0.0972,52.4000,55.1500,1.3500,1.4746,1.9081,3.6350,-0.1414,0.4211,6.2027,4.4500,-0.1491,-1.8500,-1.4000,-3.0500,-1.2000,-2.1000,-5.7000,-1.7000,-6.3000,3.3000,-0.1000,0.9500,1.1500,1.2000,0.0017,-0.0500,-0.8000,-4.9573,-8.0000,-8.5000
50%,2.0564,0.3546,0.3554,0.9610,64.9383,-3.2022,2.4576,2.5319,1.6024,-12.9539,0.1711,54.9000,56.1000,5.6000,1.6983,2.3039,3.8600,2.2457,0.5237,6.5862,4.7000,0.0747,0.4000,4.7000,3.9000,2.7000,11.1000,9.8000,6.4000,3.3000,3.5000,-0.1000,1.3000,1.4000,1.4000,0.0430,0.2000,0.2000,1.8700,3.0000,0.0000
75%,2.3418,1.5611,1.5248,1.4294,93.7296,29.1530,2.7347,2.7857,1.8734,15.2778,0.3147,56.2500,57.2000,9.1000,2.5410,3.2297,4.5500,3.5782,0.6619,6.7798,4.9000,0.2690,5.2500,5.9000,8.2000,6.2500,17.9000,15.6000,17.1000,9.0000,3.5500,0.1000,1.4000,2.1000,1.9000,0.0798,0.4500,0.5500,4.1790,8.0000,8.5000
max,2.8257,2.6782,2.4056,1.9899,97.9358,61.8537,5.8002,8.4053,5.2428,133.9864,3.0293,59.7000,62.0000,15.3000,3.1237,3.4437,5.7600,11.5399,1.1404,7.4220,8.0000,1.9324,37.6000,11.1000,22.0000,12.5000,23.2000,27.1000,22.8000,20.7000,4.2000,0.7000,5.0000,4.7000,4.0000,0.1059,1.4000,2.0000,13.7532,13.0000,11.0000


## 파일로 저장

In [19]:
# 파일로 저장
df_merge.to_csv('../data/5_데이터셋생성_재무+거시.csv', index=None)